# Here we prepair the human analys for the results of GPT for the input with textual representation and annotations

In [1]:
from support_functions.load_json_chatgpt_results import load_chat_gpt_results
from support_functions.excel_helper import prepaire_excel_data
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.data_transformations import convert_annotation_dataset

import json
import pandas as pd
import copy
from enum import Enum
from ordered_set import OrderedSet

In [2]:
class RedundancyType(Enum):
    MAIN_PART_REDUNDANCY = "mainPart"
    BENEFIT_REDUNDANCY = "benefit"
    MAIN_PART_BENEFIT_REDUNDANCY = "mainPart-benefit"

In [3]:
def clean_convertable(filtered_rows):
        convertable = json.loads(filtered_rows.to_json(orient="records"))[0]
        del convertable["Project Identifier"]
        del convertable["User Story 1 Text with Markers"]
        del convertable["User Story 2 Text with Markers"]
        del convertable["Total Redundancy Clause"]
        del convertable["USID 1"]
        del convertable["USID 2"]
        return convertable

In [4]:
def calc_repair_avg(json_data: list[dict]) -> int:
    count: int = 0
    i: int = 0
    temp: int = None
    for usid_redundancy_key, item in json_data.items():
        model = usid_redundancy_key
        for sub_key, sub_item in item.items():
            data_set = sub_key
            for _ in sub_item:
                temp = int(_["repairRuns"])
                if temp > 0:
                    count += temp
                i += 1            
    return round(count / i, 2)

In [5]:
def calc_total_repair(json_data: list[dict]) -> int:
    count: int = 0
    temp: int = None
    for usid_redundancy_key, item in json_data.items():
        model = usid_redundancy_key
        for sub_key, sub_item in item.items():
            data_set = sub_key
            for _ in sub_item:
                temp = int(_["repairRuns"])
                if temp > 0:
                    count += temp
    return count

## Getting Graph-Transformation Data

In [6]:
excel_data_graph_approach = prepaire_excel_data("Framework")
display(excel_data_graph_approach.head(3))

,Identifier,Project Identifier,USID 1,User Story 1 Text with Markers,USID 2,User Story 2 Text with Markers,Total Redundancy Clause,Main Part Redundancy Clause,Benefit Part Redundancy Clause,Main Part Partial,Main Part Full,Benefit Partial,Benefit Full
1,g19user_story_1021_AND_user_story_1036,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1036,"As an OlderPerson, I want to #use# #ALFRED# to...",2,2,0,True,False,False,False
2,g19user_story_1021_AND_user_story_1056,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1056,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False
3,g19user_story_1021_AND_user_story_1057,g19,1021,"As an OlderPerson, I want to #use# #ALFRED# as...",1057,"As an OlderPerson, I want to #use# #ALFRED# fo...",2,2,0,True,False,False,False


## Graph-Transformation Results and GPT Text Results

In [7]:
directories_to_ignore_text:list = ["text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125", 
                              "text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "text-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125",
                              "strict-annoations-redundancy-model-gpt-4o-mini-2024-07-18"]
gpt_results_text: dict[dict[str, list[dict]]] = load_chat_gpt_results(directories_to_ignore_text)
gpt_result_keys_text: list[str] = list(gpt_results_text.keys())
gpt_result_json_keys_text = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in gpt_results_text.items()
}

### Basic analyses

In [8]:
redundant_main_part_text: int = 0
redundant_benefit_text: int = 0
no_redundancies_main_part_text: int = 0
no_redundancies_benefit_text: int = 0

In [9]:
for usid_redundancy_key, item in gpt_results_text.items():
    model = usid_redundancy_key
    for sub_key, sub_item in item.items():
        data_set = sub_key
        for _ in sub_item:
            if bool(_["mainPartRedundancies"]["mainPartRedundancy"]):
                redundant_main_part_text += 1
            else:
                no_redundancies_main_part_text += 1
            if bool(_["benefitRedundancies"]["benefitRedundancy"]):
                redundant_benefit_text += 1
            else:
                no_redundancies_benefit_text += 1
        print(f"{model}, {data_set}, Strict Redundancies in Main Part: {redundant_main_part_text}, No Strict Reduncancies in Main Part: {no_redundancies_main_part_text}, Strict Redundancies in Benefit: {redundant_benefit_text}, No Strict Redundancies in Benefit: {no_redundancies_benefit_text}")
        redundant_main_part_text = no_redundancies_main_part_text = redundant_benefit_text = no_redundancies_benefit_text = 0

strict_redundancy_text-redundancy-model-gpt-4o-mini-2024-07-18, 00_g19.json, Strict Redundancies in Main Part: 1709, No Strict Reduncancies in Main Part: 7443, Strict Redundancies in Benefit: 126, No Strict Redundancies in Benefit: 9026
strict_redundancy_text-redundancy-model-gpt-4o-mini-2024-07-18, 00_g22.json, Strict Redundancies in Main Part: 227, No Strict Reduncancies in Main Part: 3174, Strict Redundancies in Benefit: 135, No Strict Redundancies in Benefit: 3266


In [10]:
print(calc_repair_avg(gpt_results_text))
print(calc_total_repair(gpt_results_text))

0.19
2407


In [11]:
not_processed_datasets_text: dict[str, list] = loading()
datasets_text, ignored_items = convert_annotation_dataset(not_processed_datasets_text)

g19_text = datasets_text["g19"]
g22_text = datasets_text["g22"]
print(g22_text)

[{'PID': '#G22#', 'USID': '1225', 'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.', 'Main Part': 'As a PI, I want to properly record all metadata', 'Benefit': 'I can ensure proper running of the project in case of staff changes', 'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []}, 'Targets': {'Main Part': [['properly record', 'all metadata']], 'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]}, 'Contains': {'Main Part': [], 'Benefit': []}}, {'PID': '#G22#', 'USID': '1226', 'Text': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project, so that it remains accessible during all stages of the data lifecycle.', 'Main Part': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project', 'Benefit': 'it remains accessible during all stages of the data lifecycle', 'Tr

### Getting overlaps between graph transformation approach and llm (just text)

In [12]:
strict_llm_results_text: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:
    tp_main_part:int = 0
    fp_main_part:int = 0
    fn_main_part:int = 0
    tn_main_part:int = 0
    
    tp_benefit:int = 0
    fp_benefit:int = 0
    fn_benefit:int = 0
    tn_benefit:int = 0
    
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp:str = None
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_text.values():
        for dataset_key, array in dataset.items():
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                
                entries_in_filtered_row = len(filtered_rows)
                if entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                temp = ""
                try:
                    if (not bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Main Part Full"])):
                        if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                            fp_main_part += 1
                            temp += "main part,fp"
                        else:
                            tn_main_part += 1
                            temp += "main part,tn"
                    else: 
                        if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                            tp_main_part += 1
                            temp += "main part,tp"
                        else:
                            fn_main_part += 1
                            temp += "main part,fn"
                    
                    temp += ","
                    
                    if (not bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Benefit Full"])):
                        if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                            fp_benefit += 1
                            temp += "benefit,fp"
                        else:
                            tn_benefit += 1
                            temp += "benefit,tn"
                    else: 
                        if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                            tp_benefit += 1
                            temp += "benefit,tp"
                        else:
                            fn_benefit += 1
                            temp += "benefit,fn"
                except:
                    if bool(llm_detection_values["mainPartRedundancies"]["mainPartRedundancy"]):
                        fp_main_part += 1
                        temp += "main part,fp"
                    else:
                        tn_main_part += 1
                        temp += "main part,tn"
                        
                    temp += ","
                    
                    if bool(llm_detection_values["benefitRedundancies"]["benefitRedundancy"]):
                        fp_benefit += 1
                        temp += "benefit,fp"
                    else:
                        tn_benefit += 1
                        temp += "benefit,tn"
                        
                if len(temp) > 0:
                    temp = f"{usid1},{usid2},{temp}"
                    strict_llm_results_text.add(temp)                                
                
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None
    print(f"Main Part: TP: {tp_main_part}, FP: {fp_main_part}, FN: {fn_main_part}, TN: {tn_main_part}")
    print(f"Benefit: TP: {tp_benefit}, FP: {fp_benefit}, FN: {fn_benefit}, TN: {tn_benefit}")
            

Main Part: TP: 525, FP: 1411, FN: 52, TN: 10565
Benefit: TP: 23, FP: 238, FN: 12, TN: 12280


In [13]:
for item in strict_llm_results_text:
    elements = item.split(",")
    #if elements[3] == "fp" or elements[5] == "fp":
    print(",".join(map(str, elements)))

1018,1019,main part,tn,benefit,tn
1018,1020,main part,tn,benefit,tn
1018,1021,main part,tn,benefit,tn
1018,1022,main part,fp,benefit,tn
1018,1023,main part,tn,benefit,tn
1018,1024,main part,tn,benefit,tn
1018,1025,main part,tn,benefit,tn
1018,1026,main part,tn,benefit,tn
1018,1027,main part,tn,benefit,tn
1018,1028,main part,tn,benefit,tn
1018,1029,main part,tn,benefit,tn
1018,1030,main part,tn,benefit,tn
1018,1031,main part,tn,benefit,tn
1018,1032,main part,tn,benefit,tn
1018,1033,main part,tn,benefit,tn
1018,1034,main part,tn,benefit,tn
1018,1035,main part,tn,benefit,tn
1018,1036,main part,tn,benefit,tn
1018,1037,main part,tn,benefit,tn
1018,1038,main part,tn,benefit,tn
1018,1039,main part,tn,benefit,tn
1018,1040,main part,tn,benefit,tn
1018,1041,main part,fp,benefit,tn
1018,1042,main part,tn,benefit,tn
1018,1043,main part,tn,benefit,tn
1018,1044,main part,tn,benefit,tn
1018,1045,main part,tn,benefit,tn
1018,1046,main part,tn,benefit,tn
1018,1047,main part,fp,benefit,tn
1018,1048,main

## Graph-Transformation-Results and GPT-Annotation Results

In [14]:
directories_to_ignore_annotations:list[str] = ["text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125", 
                              "text-annoations-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-4o", 
                              "text-redundancy-model-gpt-4o-mini-2024-07-18", "annoations-redundancy-model-gpt-3.5-turbo-0125",
                              "strict_redundancy_text-redundancy-model-gpt-4o-mini-2024-07-18"]
gpt_results_annotations: dict[dict[str, list[dict]]] = load_chat_gpt_results(directories_to_ignore_annotations)
gpt_result_keys_annotations: list[str] = list(gpt_results_annotations.keys())
gpt_result_json_keys_annotations = {
    outer_key: list(inner_dict.keys())
    for outer_key, inner_dict in gpt_results_annotations.items()
}

### Basic analyses

In [15]:
redundant_main_part_annotations: int = 0
redundant_benefit_annotations: int = 0
no_redundancies_main_part_annotations: int = 0
no_redundancies_benefit_annotations: int = 0

In [16]:
for usid_redundancy_key, item in gpt_results_annotations.items():
    model = usid_redundancy_key
    for sub_key, sub_item in item.items():
        data_set = sub_key
        for _ in sub_item:
            if (bool(_["mainPartRedundancies"]["fullRedundancy"]) 
                    or bool(_["mainPartRedundancies"]["partialRedundancy"])):
                redundant_main_part_annotations += 1
            else:
                no_redundancies_main_part_annotations += 1
            if (bool(_["benefitRedundancies"]["fullRedundancy"]) 
                    or bool(_["benefitRedundancies"]["partialRedundancy"])):
                redundant_benefit_annotations += 1
            else:
                no_redundancies_benefit_annotations += 1
        print(f"{model}, {data_set}, Strict Redundancies in Main Part: {redundant_main_part_annotations}, No Strict Reduncancies in Main Part: {no_redundancies_main_part_annotations}, Strict Redundancies in Benefit: {redundant_benefit_annotations}, No Strict Redundancies in Benefit: {no_redundancies_benefit_annotations}")
        redundant_main_part_annotations = no_redundancies_main_part_annotations = redundant_benefit_annotations = no_redundancies_benefit_annotations = 0

strict-annoations-redundancy-model-gpt-4o-mini-2024-07-18, 00_g19.json, Strict Redundancies in Main Part: 1122, No Strict Reduncancies in Main Part: 8131, Strict Redundancies in Benefit: 122, No Strict Redundancies in Benefit: 9131
strict-annoations-redundancy-model-gpt-4o-mini-2024-07-18, 00_g22.json, Strict Redundancies in Main Part: 252, No Strict Reduncancies in Main Part: 3125, Strict Redundancies in Benefit: 90, No Strict Redundancies in Benefit: 3287


In [17]:
print(calc_repair_avg(gpt_results_annotations))
print(calc_total_repair(gpt_results_annotations))

0.03
442


### Get all Redundant US with text (add to json)

In [18]:
not_processed_datasets_annotations: dict[str, list] = loading()
datasets_annotations, ignored_items = convert_annotation_dataset(not_processed_datasets_annotations)

g19_annotations = datasets_annotations["g19"]
g22_annotations = datasets_annotations["g22"]
print(g22_annotations)

[{'PID': '#G22#', 'USID': '1225', 'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.', 'Main Part': 'As a PI, I want to properly record all metadata', 'Benefit': 'I can ensure proper running of the project in case of staff changes', 'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []}, 'Targets': {'Main Part': [['properly record', 'all metadata']], 'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]}, 'Contains': {'Main Part': [], 'Benefit': []}}, {'PID': '#G22#', 'USID': '1226', 'Text': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project, so that it remains accessible during all stages of the data lifecycle.', 'Main Part': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project', 'Benefit': 'it remains accessible during all stages of the data lifecycle', 'Tr

## Getting the non-/overlaps between graph transformation approach and llm

In [19]:
strict_llm_results_annotations: OrderedSet = OrderedSet() # value is the pair numbers with redundancy type

if True:
    tp_main_part:int = 0
    fp_main_part:int = 0
    fn_main_part:int = 0
    tn_main_part:int = 0
    
    tp_benefit:int = 0
    fp_benefit:int = 0
    fn_benefit:int = 0
    tn_benefit:int = 0
    
    
    filtered_rows: pd.DataFrame = None
    entries_in_filtered_row: int = 0
    usid1: int = 0
    usid2: int = 0
    usid_redundancy_key: str = ""
    temp_list: list[dict[str, dict]] = []
    convertable_gt_system_output: dict = {}
    for dataset in gpt_results_annotations.values():
        for dataset_key, array in dataset.items():
            for llm_detection_values in array:
                usid1, usid2 = llm_detection_values["relatedStories"]
                filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 1'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 2'].astype(int) == int(usid2))
                    ]
                if filtered_rows.empty:
                    filtered_rows = excel_data_graph_approach[
                        (excel_data_graph_approach['USID 2'].astype(int) == int(usid1)) 
                        & (excel_data_graph_approach['USID 1'].astype(int) == int(usid2))
                    ]
                    
                entries_in_filtered_row = len(filtered_rows)
                if  entries_in_filtered_row > 1:
                    raise ValueError(f"For {item[0]}, multiple entries in the excel file were found.")
                
                temp = ""
                try:
                    if (not bool(filtered_rows.iloc[0]["Main Part Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Main Part Full"])):
                        if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            fp_main_part += 1
                            temp += "main part,fp"
                        else:
                            tn_main_part += 1
                            temp += "main part,tn"
                    else: 
                        if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            tp_main_part += 1
                            temp += "main part,tp"
                        else:
                            fn_main_part += 1
                            temp += "main part,fn"
                    
                    temp += ","
                    
                    
                    if (not bool(filtered_rows.iloc[0]["Benefit Partial"]) 
                        and not bool(filtered_rows.iloc[0]["Benefit Full"])):
                        if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                            fp_benefit += 1
                            temp += "benefit,fp"
                        else:
                            tn_benefit += 1
                            temp += "benefit,tn"
                    else: 
                        if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                            tp_benefit += 1
                            temp += "benefit,tp"
                        else:
                            fn_benefit += 1
                            temp += "benefit,fn"
                except:
                    if (bool(llm_detection_values["mainPartRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["mainPartRedundancies"]["fullRedundancy"])):
                            fp_main_part += 1
                            temp += "main part,fp"
                    else:
                        tn_main_part += 1
                        temp += "main part,tn"
                    temp += ","
                    if (bool(llm_detection_values["benefitRedundancies"]["partialRedundancy"])
                                or bool(llm_detection_values["benefitRedundancies"]["fullRedundancy"])):
                        fp_benefit += 1
                        temp += "benefit,fp"
                    else:
                        tn_benefit += 1
                        temp += "benefit,tn"
                        
                if len(temp) > 0:
                    temp = f"{usid1},{usid2},{temp}"
                    strict_llm_results_annotations.add(temp)                                
                
                filtered_rows = None
                entries_in_filtered_row = None
                usid1 = None
                usid2 = None
                usid_redundancy_key = None
            temp_list = None
    print(f"Main Part: TP: {tp_main_part}, FP: {fp_main_part}, FN: {fn_main_part}, TN: {tn_main_part}")
    print(f"Benefit: TP: {tp_benefit}, FP: {fp_benefit}, FN: {fn_benefit}, TN: {tn_benefit}")

Main Part: TP: 557, FP: 817, FN: 3, TN: 11253
Benefit: TP: 32, FP: 180, FN: 2, TN: 12416


In [20]:
for item in strict_llm_results_annotations:
    elements = item.split(",")
    #if elements[3] == "fp" or elements[5] == "fp":
    print(",".join(map(str, elements)))

1018,1019,main part,tn,benefit,tn
1018,1020,main part,tn,benefit,tn
1018,1021,main part,tn,benefit,tn
1018,1022,main part,tn,benefit,tn
1018,1023,main part,tn,benefit,tn
1018,1024,main part,tn,benefit,tn
1018,1025,main part,tn,benefit,tn
1018,1028,main part,fp,benefit,tn
1018,1029,main part,tn,benefit,tn
1018,1030,main part,tn,benefit,tn
1018,1031,main part,tn,benefit,tn
1018,1032,main part,tn,benefit,tn
1018,1033,main part,tn,benefit,tn
1018,1034,main part,tn,benefit,tn
1018,1035,main part,tn,benefit,tn
1018,1036,main part,tn,benefit,tn
1018,1037,main part,tn,benefit,tn
1018,1038,main part,fp,benefit,tn
1018,1039,main part,tn,benefit,tn
1018,1040,main part,tn,benefit,tn
1018,1041,main part,tn,benefit,tn
1018,1042,main part,tn,benefit,tn
1018,1043,main part,tn,benefit,tn
1018,1044,main part,tn,benefit,tn
1018,1045,main part,tn,benefit,tn
1018,1046,main part,tn,benefit,tn
1018,1047,main part,tn,benefit,tn
1018,1048,main part,tn,benefit,tn
1018,1049,main part,tn,benefit,tn
1018,1050,main